In [3]:
!aws s3 ls s3://amazon-reviews-pds/tsv/

2017-11-24 13:22:50          0 
2017-11-24 13:48:03  241896005 amazon_reviews_multilingual_DE_v1_00.tsv.gz
2017-11-24 13:48:17   70583516 amazon_reviews_multilingual_FR_v1_00.tsv.gz
2017-11-24 13:48:34   94688992 amazon_reviews_multilingual_JP_v1_00.tsv.gz
2017-11-24 13:49:14  349370868 amazon_reviews_multilingual_UK_v1_00.tsv.gz
2017-11-24 13:48:47 1466965039 amazon_reviews_multilingual_US_v1_00.tsv.gz
2017-11-24 13:49:53  648641286 amazon_reviews_us_Apparel_v1_00.tsv.gz
2017-11-24 13:56:36  582145299 amazon_reviews_us_Automotive_v1_00.tsv.gz
2017-11-24 14:04:02  357392893 amazon_reviews_us_Baby_v1_00.tsv.gz
2017-11-24 14:08:11  914070021 amazon_reviews_us_Beauty_v1_00.tsv.gz
2017-11-24 14:17:41 2740337188 amazon_reviews_us_Books_v1_00.tsv.gz
2017-11-24 14:45:50 2692708591 amazon_reviews_us_Books_v1_01.tsv.gz
2017-11-24 15:10:21 1329539135 amazon_reviews_us_Books_v1_02.tsv.gz
2017-11-24 15:22:13  442653086 amazon_reviews_us_Camera_v1_00.tsv.gz
2017-11-24 15:27:13 2689739299 amazon_rev

In [4]:
!aws s3 ls s3://amazon-reviews-pds/parquet/

                           PRE product_category=Apparel/
                           PRE product_category=Automotive/
                           PRE product_category=Baby/
                           PRE product_category=Beauty/
                           PRE product_category=Books/
                           PRE product_category=Camera/
                           PRE product_category=Digital_Ebook_Purchase/
                           PRE product_category=Digital_Music_Purchase/
                           PRE product_category=Digital_Software/
                           PRE product_category=Digital_Video_Download/
                           PRE product_category=Digital_Video_Games/
                           PRE product_category=Electronics/
                           PRE product_category=Furniture/
                           PRE product_category=Gift_Card/
                           PRE product_category=Grocery/
                           PRE product_category=Health_&_Personal_Care/
   

In [3]:
import boto3
import sagemaker
import pandas as pd
import awswrangler as wr

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

In [23]:
!aws s3 mb s3://amazon-reviews-capstone

make_bucket: amazon-reviews-capstone


In [25]:
s3_public_source_tsv = "s3://amazon-reviews-pds/tsv"

In [26]:
%store s3_source_tsv

Stored 's3_source_tsv' (str)


In [27]:
s3_private_source_tsv = "s3://amazon-reviews-capstone/amazon-reviews-pds/tsv".format(bucket)
print(s3_private_source_tsv)

s3://amazon-reviews-capstone/amazon-reviews-pds/tsv


In [28]:
%store s3_private_source_tsv

Stored 's3_private_source_tsv' (str)


In [29]:
!aws s3 cp --recursive $s3_public_source_tsv/ $s3_private_source_tsv/

copy: s3://amazon-reviews-pds/tsv/amazon_reviews_multilingual_FR_v1_00.tsv.gz to s3://amazon-reviews-capstone/amazon-reviews-pds/tsv/amazon_reviews_multilingual_FR_v1_00.tsv.gz
copy: s3://amazon-reviews-pds/tsv/amazon_reviews_multilingual_JP_v1_00.tsv.gz to s3://amazon-reviews-capstone/amazon-reviews-pds/tsv/amazon_reviews_multilingual_JP_v1_00.tsv.gz
copy: s3://amazon-reviews-pds/tsv/amazon_reviews_us_Apparel_v1_00.tsv.gz to s3://amazon-reviews-capstone/amazon-reviews-pds/tsv/amazon_reviews_us_Apparel_v1_00.tsv.gz
copy: s3://amazon-reviews-pds/tsv/amazon_reviews_multilingual_DE_v1_00.tsv.gz to s3://amazon-reviews-capstone/amazon-reviews-pds/tsv/amazon_reviews_multilingual_DE_v1_00.tsv.gz
copy: s3://amazon-reviews-pds/tsv/amazon_reviews_multilingual_UK_v1_00.tsv.gz to s3://amazon-reviews-capstone/amazon-reviews-pds/tsv/amazon_reviews_multilingual_UK_v1_00.tsv.gz
copy: s3://amazon-reviews-pds/tsv/amazon_reviews_us_Beauty_v1_00.tsv.gz to s3://amazon-reviews-capstone/amazon-reviews-pds/ts

In [38]:
import sys
!{sys.executable} -m pip install PyAthena
db_name = "amazon_reviews_capstone"


[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [40]:
from pyathena import connect
import pandas as pd
conn = connect(s3_staging_dir = "s3://{0}/athena/staging".format(bucket),
               region_name="us-east-1")



In [41]:
statement = "CREATE DATABASE IF NOT EXISTS {}".format(db_name)
print(statement)

CREATE DATABASE IF NOT EXISTS amazon_reviews_capstone


In [42]:
import pandas as pd

pd.read_sql(statement, conn)

""


In [225]:
table_name_tsv = "amazon_reviews_tsv"

# SQL statement to execute
statement1 = """CREATE EXTERNAL TABLE IF NOT EXISTS amazon_reviews_capstone.amazon_reviews_tsv(
    marketplace string,
    customer_id string,
    review_id string,
    product_id string,
    product_parent string,
    product_title string,
    product_category string,
    star_rating int,
    helpful_votes int,
    total_votes int,
    vine string,
    verified_purchase string,
    review_headline string,
    review_body string,
    review_date string
    ) 
    ROW FORMAT DELIMITED FIELDS TERMINATED BY '\t'
    LINES TERMINATED BY '\n'
    LOCATION 's3://amazon-reviews-capstone/amazon-reviews-pds/tsv'
    TBLPROPERTIES ('compressionType'='gzip', 'skip.header.line.count'='1')"""

pd.read_sql(statement1, conn)
print(statement1)

CREATE EXTERNAL TABLE IF NOT EXISTS amazon_reviews_capstone.amazon_reviews_tsv(
    marketplace string,
    customer_id string,
    review_id string,
    product_id string,
    product_parent string,
    product_title string,
    product_category string,
    star_rating int,
    helpful_votes int,
    total_votes int,
    vine string,
    verified_purchase string,
    review_headline string,
    review_body string,
    review_date string
    ) 
    ROW FORMAT DELIMITED FIELDS TERMINATED BY '	'
    LINES TERMINATED BY '
'
    LOCATION 's3://amazon-reviews-capstone/amazon-reviews-pds/tsv'
    TBLPROPERTIES ('compressionType'='gzip', 'skip.header.line.count'='1')


In [82]:
df = pd.read_sql("SELECT * FROM amazon_reviews_capstone.amazon_reviews_tsv limit 8;", conn)
df

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,35471030,R1C5OK9AND7PRP,B00AHK07V0,175130663,Hunter's Moon (A Kate Shugak Investigation Boo...,Digital_Ebook_Purchase,5,0,0,N,Y,"Shugak is like a milder, more withdrawn Reacher",This is the most intense of the Kate Shugak se...,2015-08-31
1,US,26579324,R333RNBQMPVUFT,B014085WTQ,859232728,Flying Toward Forgiveness,Digital_Ebook_Purchase,5,0,0,N,Y,"For the young heart that, ages later, asks ""wh...",This story holds so much more than meets the e...,2015-08-31
2,US,9152364,R2A2K0GTNSKXM1,B0113MFPN0,411472516,Falling For A Real Nigga,Digital_Ebook_Purchase,5,0,0,N,N,Five Stars,A true love story I think.,2015-08-31
3,US,52087605,R365LAQ9REV876,B00T8RIK7G,353828021,The Good Neighbor,Digital_Ebook_Purchase,3,0,0,N,Y,Not really my type of read,A bit contrived and weak for my liking. I did ...,2015-08-31
4,US,46454660,R294OF3SNH6SWZ,B00EMXBDMA,803172158,The Martian: A Novel,Digital_Ebook_Purchase,4,0,0,N,Y,Coming soon to a theater near you,Moved well. Seemed almost like a sketch for a ...,2015-08-31
5,US,33168429,R2OMRH58WYEEP9,B00GL3OJIY,325772386,The Painter: A novel,Digital_Ebook_Purchase,5,0,0,N,N,I liked it a,Very different and extremely well written; I l...,2015-08-31
6,US,18546798,RBFXYUJZIH5JR,B00RW30QE8,236508296,METEOR STORM,Digital_Ebook_Purchase,5,0,0,N,Y,The author delivered his promise for an intere...,Once I started reading this novel I could not ...,2015-08-31
7,US,47463409,R2K2RSBNV42HU4,B0101AE85Q,924272105,Diary of a Crazy Steve [An Unofficial MineCraf...,Digital_Ebook_Purchase,5,0,0,N,N,This book is amazing.,"Super quality, great grammar, no spelling issu...",2015-08-31


In [83]:
table_name_parquet = "amazon_reviews_parquet"
# Set S3 path to Parquet data
s3_path_parquet = ç
# SQL statement to execute
statement2 = """CREATE TABLE IF NOT EXISTS amazon_reviews_capstone.amazon_reviews_parquet
WITH (format = 'PARQUET', external_location = 's3://amazon-reviews-capstone/amazon-reviews-pds/parquet', partitioned_by = ARRAY['product_category']) AS
SELECT marketplace,
         customer_id,
         review_id,
         product_id,
         product_parent,
         product_title,
         star_rating,
         helpful_votes,
         total_votes,
         vine,
         verified_purchase,
         review_headline,
         review_body,
         CAST(YEAR(DATE(review_date)) AS INTEGER) AS year,
         DATE(review_date) AS review_date,
         product_category
FROM amazon_reviews_capstone.amazon_reviews_tsv"""
pd.read_sql(statement2, conn)
print(statement2)

CREATE TABLE IF NOT EXISTS amazon_reviews_capstone.amazon_reviews_parquet
WITH (format = 'PARQUET', external_location = 's3://amazon-reviews-capstone/amazon-reviews-pds/parquet', partitioned_by = ARRAY['product_category']) AS
SELECT marketplace,
         customer_id,
         review_id,
         product_id,
         product_parent,
         product_title,
         star_rating,
         helpful_votes,
         total_votes,
         vine,
         verified_purchase,
         review_headline,
         review_body,
         CAST(YEAR(DATE(review_date)) AS INTEGER) AS year,
         DATE(review_date) AS review_date,
         product_category
FROM amazon_reviews_capstone.amazon_reviews_tsv


In [85]:
statement = "MSCK REPAIR TABLE amazon_reviews_capstone.amazon_reviews_parquet".format(db_name, table_name_parquet)

print(statement)

MSCK REPAIR TABLE amazon_reviews_capstone.amazon_reviews_parquet


In [87]:
df = pd.read_sql(statement1, conn)
df.head(100)

""


In [77]:
statement = "SHOW PARTITIONS amazon_reviews_capstone.amazon_reviews_parquet".format(db_name, table_name_parquet)

print(statement)

SHOW PARTITIONS amazon_reviews_capstone.amazon_reviews_parquet


In [79]:
df_partitions = pd.read_sql(statement2, conn)
df_partitions.head(100)

,partition
0,product_category=Grocery
1,product_category=Video
2,product_category=Video DVD
3,product_category=Home
4,product_category=Home Entertainment
5,product_category=Pet Products
6,product_category=Digital_Video_Download
7,product_category=Toys
8,product_category=Beauty
9,product_category=Automotive


In [5]:
p_filter = lambda x: x["product_category"] == "Digital_Software"

In [6]:
path = "s3://amazon-reviews-capstone/amazon-reviews-pds/parquet/".format(bucket)
df_parquet_results = wr.s3.read_parquet(
    path, columns=["star_rating", "product_category", "review_body"], partition_filter=p_filter, dataset=True
)
df_parquet_results.shape

(102084, 3)

In [7]:
df_parquet_results.head(5)

,star_rating,review_body,product_category
0,4,it works,Digital_Software
1,4,good product,Digital_Software
2,5,I have been using Video Converter for about on...,Digital_Software
3,4,This was just an update.,Digital_Software
4,4,Worked just fine. I will use again next year!,Digital_Software


In [8]:
chunk_iter = wr.s3.read_parquet(
    path,
    columns=["star_rating", "product_category", "review_body"],
    # filters=[("product_category", "=", "Digital_Software")],
    partition_filter=p_filter,
    dataset=True,
    chunked=True,
)

In [9]:
print(next(chunk_iter))

      star_rating                                        review_body  \
0               4                                           it works   
1               4                                       good product   
2               5  I have been using Video Converter for about on...   
3               4                           This was just an update.   
4               4     Worked just fine.  I will use again next year!   
...           ...                                                ...   
3724            1  I downloaded this version of Quicken expecting...   
3725            5  Must have if you're a photographer.  This stre...   
3726            5  I have been using TT for well over 15 years no...   
3727            5  First of all, Amazon makes the downloading of ...   
3728            1  The amount of hours I have spent, doing what c...   

      product_category  
0     Digital_Software  
1     Digital_Software  
2     Digital_Software  
3     Digital_Software  
4     Digi

In [10]:
database_name = "amazon_reviews_capstone"
table_name_tsv = "amazon_reviews_tsv"
table_name_parquet = "amazon_reviews_parquet"

In [11]:
for table in wr.catalog.get_tables(database="dsoaws"):
    print(table["Name"])

amazon_reviews_parquet
amazon_reviews_tsv


In [16]:
%%time
df = wr.athena.read_sql_query(sql="SELECT * FROM amazon_reviews_capstone.amazon_reviews_parquet LIMIT 5000".format(table_name_parquet), database=database_name)

CPU times: user 425 ms, sys: 9.76 ms, total: 435 ms
Wall time: 3.63 s


In [18]:
df.head(5)

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,28794885,REAKC26P07MDN,B00Q0K9604,510387886,(8-Pack) EZwhelp Belly Band/Wrap,Pet Products,5,0,0,N,Y,"A great purchase for ""dribbly"" dogs",Best belly bands on the market! These are a g...,2015-08-31
1,US,11488901,R3NU7OMZ4HQIEG,B00MBW5O9W,912374672,Warren Eckstein's Hugs & Kisses Vitamin Minera...,Pet Products,2,0,1,N,Y,My dogs love Hugs and Kisses,"My dogs love Hugs and Kisses. However, the la...",2015-08-31
2,US,43214993,R14QJW3XF8QO1P,B0084OHUIO,902215727,Tyson's True Chews Premium Jerky - 12 ounce Ch...,Pet Products,5,0,0,N,Y,I have been purchasing these for a long time. ...,I have been purchasing these for a long time. ...,2015-08-31
3,US,12835065,R2HB7AX0394ZGY,B001GS71K2,568880110,"Soft Side Pet Crate, Navy/Tan",Pet Products,5,0,0,N,Y,it is easy to open and close,"It is extremely well constructed, it is easy t...",2015-08-31
4,US,26334022,RGKMPDQGSAHR3,B004ABH1LG,692846826,"EliteField 3-Door Folding Soft Dog Crate, Indo...",Pet Products,5,0,0,N,Y,Dog crate,Worked really well. Very pleased with my purc...,2015-08-31


In [20]:
%%time

chunk_iter = wr.athena.read_sql_query(
    sql="SELECT * FROM amazon_reviews_capstone.amazon_reviews_parquet LIMIT 5000".format(table_name_parquet),
    database="{}".format(database_name),
    chunksize=64_000,  # 64 KB Chunks
)

CPU times: user 297 ms, sys: 5.34 ms, total: 302 ms
Wall time: 3.67 s


In [21]:
print(next(chunk_iter))

     marketplace customer_id       review_id  product_id product_parent  \
0             US    17074612  R3GSNZVILB7N3V  B00H9L7VIW      851045898   
1             US    35812705  R2MJNGZZEHD1UD  B002ZOUP2C       15558261   
2             US     9852377  R1FWJHVUEF5NK3  B0006VJ6TO      412568457   
3             US    16122970  R1N0L1SL5JR8AU  B00H9L7VIW      851045898   
4             US     2076770  R1PV2GU6KMUGCH  B00KC3IKB6      711045215   
...          ...         ...             ...         ...            ...   
4995          US    52313599  R2VPRD92G2QCWZ  B00061OGLC      844427792   
4996          US    38772263  R27O60I47Z0MF3  B000G3DY7M      559863262   
4997          US    38497879  R28WOK321PWKK0  B000JEQPNI      430685866   
4998          US    48379291   R22UM9VS01MSP  B0013U9B6U      978181853   
4999          US    49588410  R2B6LOU7CQDODN  B005OE0VOM       71112980   

                                          product_title  star_rating  \
0     boostULTIMATE - 60 Ca

In [94]:
from botocore.exceptions import ClientError
from botocore.config import Config

In [95]:
config = Config(
   retries = {
      'max_attempts': 10,
      'mode': 'adaptive'
   }
)


iam = boto3.client('iam', config=config)
sts = boto3.client('sts')
redshift = boto3.client('redshift')
sm = boto3.client('sagemaker')
ec2 = boto3.client('ec2')

In [96]:
assume_role_policy_doc = {
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "redshift.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }
  ]
} 

In [97]:
iam_redshift_role_name = 'Amazon_Reviews_Capstone_Redshift'

In [99]:
import json
try:
    iam_role_redshift = iam.create_role(
        RoleName=iam_redshift_role_name,
        AssumeRolePolicyDocument=json.dumps(assume_role_policy_doc),
        Description='Amazon_Reviews_Capstone_Redshift'
    )
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        print("Role already exists")
    else:
        print("Unexpected error: %s" % e)

Role already exists


In [100]:
role = iam.get_role(RoleName='Amazon_Reviews_Capstone_Redshift')
iam_role_redshift_arn = role['Role']['Arn']
print(iam_role_redshift_arn)

arn:aws:iam::518606024901:role/Amazon_Reviews_Capstone_Redshift


In [101]:
account_id = sts.get_caller_identity()['Account']
print(account_id)

518606024901


In [102]:
my_redshift_to_s3 = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "s3:*",
            "Resource": "*"
        }
    ]
}

In [103]:
my_redshift_to_athena = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "athena:*"
            ],
            "Resource": [
                "*"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "glue:CreateDatabase",
                "glue:DeleteDatabase",
                "glue:GetDatabase",
                "glue:GetDatabases",
                "glue:UpdateDatabase",
                "glue:CreateTable",
                "glue:DeleteTable",
                "glue:BatchDeleteTable",
                "glue:UpdateTable",
                "glue:GetTable",
                "glue:GetTables",
                "glue:BatchCreatePartition",
                "glue:CreatePartition",
                "glue:DeletePartition",
                "glue:BatchDeletePartition",
                "glue:UpdatePartition",
                "glue:GetPartition",
                "glue:GetPartitions",
                "glue:BatchGetPartition"
            ],
            "Resource": [
                "*"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "s3:GetBucketLocation",
                "s3:GetObject",
                "s3:ListBucket",
                "s3:ListBucketMultipartUploads",
                "s3:ListMultipartUploadParts",
                "s3:AbortMultipartUpload",
                "s3:CreateBucket",
                "s3:PutObject"
            ],
            "Resource": [
                "arn:aws:s3:::aws-athena-query-results-*"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::athena-examples*"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "s3:ListBucket",
                "s3:GetBucketLocation",
                "s3:ListAllMyBuckets"
            ],
            "Resource": [
                "*"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "sns:ListTopics",
                "sns:GetTopicAttributes"
            ],
            "Resource": [
                "*"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "cloudwatch:PutMetricAlarm",
                "cloudwatch:DescribeAlarms",
                "cloudwatch:DeleteAlarms"
            ],
            "Resource": [
                "*"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "lakeformation:GetDataAccess"
            ],
            "Resource": [
                "*"
            ]
        }
    ]
}

In [104]:
my_redshift_to_sagemaker = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "sagemaker:*",
            "Resource": "*"
        }
    ]
}

In [105]:
my_redshift_to_sagemaker_passrole = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "iam:PassRole",
            "Resource": f'arn:aws:iam::{account_id}:role/*'
        }
    ]
}

In [106]:
try:
    policy_redshift_s3 = iam.create_policy(
      PolicyName='Amazon_Reviews_Capstone_Redshift_RedshiftPolicyToS3',
      PolicyDocument=json.dumps(my_redshift_to_s3)
    )
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        print("Policy already exists")
    else:
        print("Unexpected error: %s" % e)

In [110]:
# Get ARN
policy_redshift_s3_arn = f'arn:aws:iam::{account_id}:policy/Amazon_Reviews_Capstone_Redshift_RedshiftPolicyToS3'
print(policy_redshift_s3_arn)

arn:aws:iam::518606024901:policy/Amazon_Reviews_Capstone_Redshift_RedshiftPolicyToS3


In [112]:
try:
    policy_redshift_athena = iam.create_policy(
      PolicyName='Amazon_Reviews_Capstone_Redshift_RedshiftPolicyToAthena',
      PolicyDocument=json.dumps(my_redshift_to_athena)
    )
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        print("Policy already exists")
    else:
        print("Unexpected error: %s" % e)

Policy already exists


In [113]:
# Get ARN
policy_redshift_athena_arn = f'arn:aws:iam::{account_id}:policy/Amazon_Reviews_Capstone_Redshift_RedshiftPolicyToAthena'
print(policy_redshift_athena_arn)

arn:aws:iam::518606024901:policy/Amazon_Reviews_Capstone_Redshift_RedshiftPolicyToAthena


In [114]:
try:
    policy_redshift_sagemaker = iam.create_policy(
      PolicyName='Amazon_Reviews_Capstone_Redshift_RedshiftPolicyToSageMaker',
      PolicyDocument=json.dumps(my_redshift_to_sagemaker)
    )
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        print("Policy already exists")
    else:
        print("Unexpected error: %s" % e)

In [115]:
# Get ARN
policy_redshift_sagemaker_arn = f'arn:aws:iam::{account_id}:policy/Amazon_Reviews_Capstone_Redshift_RedshiftPolicyToSageMaker'
print(policy_redshift_sagemaker_arn)

arn:aws:iam::518606024901:policy/Amazon_Reviews_Capstone_Redshift_RedshiftPolicyToSageMaker


In [116]:
try:
    policy_redshift_sagemaker_passrole = iam.create_policy(
      PolicyName='Amazon_Reviews_Capstone_Redshift_RedshiftPolicyToSageMakerPassRole',
      PolicyDocument=json.dumps(my_redshift_to_sagemaker_passrole)
    )
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        print("Policy already exists")
    else:
        print("Unexpected error: %s" % e)

In [117]:
# Get ARN
policy_redshift_sagemaker_passrole_arn = f'arn:aws:iam::{account_id}:policy/Amazon_Reviews_Capstone_Redshift_RedshiftPolicyToSageMakerPassRole'
print(policy_redshift_sagemaker_passrole_arn)

arn:aws:iam::518606024901:policy/Amazon_Reviews_Capstone_Redshift_RedshiftPolicyToSageMakerPassRole


In [118]:
# Attach DSOAWS_RedshiftPolicyToAthena policy
try:
    response = iam.attach_role_policy(
        PolicyArn=policy_redshift_athena_arn,
        RoleName=iam_redshift_role_name
    )
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        print("Policy is already attached. This is ok.")
    else:
        print("Unexpected error: %s" % e)

In [119]:
# Attach DSOAWS_RedshiftPolicyToS3 policy
try:
    response = iam.attach_role_policy(
        PolicyArn=policy_redshift_s3_arn,
        RoleName=iam_redshift_role_name
    )
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        print("Policy is already attached. This is ok.")
    else:
        print("Unexpected error: %s" % e)
        

In [120]:
# Attach DSOAWS_RedshiftPolicyToSageMaker policy
try:
    response = iam.attach_role_policy(
        PolicyArn=policy_redshift_sagemaker_arn,
        RoleName=iam_redshift_role_name
    )
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        print("Policy is already attached. This is ok.")
    else:
        print("Unexpected error: %s" % e)

In [121]:
# Attach DSOAWS_RedshiftPolicyToSageMakerPassRole policy
try:
    response = iam.attach_role_policy(
        PolicyArn=policy_redshift_sagemaker_passrole_arn,
        RoleName=iam_redshift_role_name
    )
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        print("Policy is already attached. This is ok.")
    else:
        print("Unexpected error: %s" % e)

In [122]:
my_redshift_to_sagemaker_assumerole = {
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "redshift.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    },
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "sagemaker.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }
  ]
}  

In [123]:
try:
    response = iam.update_assume_role_policy(
        PolicyDocument=json.dumps(my_redshift_to_sagemaker_assumerole),
        RoleName=iam_redshift_role_name
    )
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        print("Policy is already attached. This is ok.")
    else:
        print("Unexpected error: %s" % e)

In [124]:
try:
    domain_id = sm.list_domains()['Domains'][0]['DomainId']
    describe_domain_response = sm.describe_domain(DomainId=domain_id)
    vpc_id = describe_domain_response['VpcId']
    security_groups = ec2.describe_security_groups()['SecurityGroups']
    for security_group in security_groups:
        if vpc_id == security_group['VpcId']:
            security_group_id = security_group['GroupId']
    print(security_group_id)    
except:
    pass

sg-06f6b0e4bebaf0b16


In [125]:
try:
    notebook_instance_name = sm.list_notebook_instances()['NotebookInstances'][0]['NotebookInstanceName']
    notebook_instance = sm.describe_notebook_instance(NotebookInstanceName=notebook_instance_name)
    security_group_id = notebook_instance['SecurityGroups'][0]
    print(security_group_id)    
except:
    pass

In [126]:
secretsmanager = boto3.client('secretsmanager')

try:
    response = secretsmanager.create_secret(
        Name='dsoaws_redshift_login',
        Description='DSOAWS Redshift Login',
        SecretString='[{"username":"dsoaws"},{"password":"Password9"}]',
        Tags=[
            {
                'Key': 'name',
                'Value': 'dsoaws_redshift_login'
            },
        ]
    )
except ClientError as e:
    if e.response['Error']['Code'] == 'ResourceExistsException':
        print("Secret already exists. This is ok.")
    else:
        print("Unexpected error: %s" % e)

Secret already exists. This is ok.


In [127]:
secret = secretsmanager.get_secret_value(SecretId='dsoaws_redshift_login')
cred = json.loads(secret['SecretString'])

master_user_name = cred[0]['username']
master_user_pw = cred[1]['password']


In [137]:
# Redshift configuration parameters
redshift_cluster_identifier = 'amazonreviewscluster'
db_name = 'amazon_reviews_capstone'
cluster_type = 'multi-node'

# Note that only some Instance Types support Redshift Query Editor 
# (https://docs.aws.amazon.com/redshift/latest/mgmt/query-editor.html)
node_type = 'dc2.large'
number_nodes = '2' 

In [138]:
response = redshift.create_cluster(
        DBName=db_name,
        ClusterIdentifier=redshift_cluster_identifier,
        ClusterType=cluster_type,
        NodeType=node_type,
        NumberOfNodes=int(number_nodes),       
        MasterUsername=master_user_name,
        MasterUserPassword=master_user_pw,
        IamRoles=[iam_role_redshift_arn],
        VpcSecurityGroupIds=['sg-0ec43a40a91763ece'],
        Port=5439,
        PubliclyAccessible=False
)


In [139]:
import time

response = redshift.describe_clusters(ClusterIdentifier=redshift_cluster_identifier)
cluster_status = response['Clusters'][0]['ClusterStatus']
print(cluster_status)

while cluster_status != 'available':
    time.sleep(10)
    response = redshift.describe_clusters(ClusterIdentifier=redshift_cluster_identifier)
    cluster_status = response['Clusters'][0]['ClusterStatus']
    print(cluster_status)

creating
creating
creating
creating
creating
creating
creating
creating
available


In [140]:
redshift_endpoint_address = response['Clusters'][0]['Endpoint']['Address']
iam_role = response['Clusters'][0]['IamRoles'][0]['IamRoleArn']

print('Redshift endpoint: {}'.format(redshift_endpoint_address))
print('IAM Role: {}'.format(iam_role))

Redshift endpoint: amazonreviewscluster.cqj1po1m4uti.us-east-1.redshift.amazonaws.com
IAM Role: arn:aws:iam::518606024901:role/Amazon_Reviews_Capstone_Redshift


In [150]:
database_name_redshift = 'amazon_reviews_capstone'
database_name_athena = 'amazon_reviews_capstone'
region_name = 'us-east-1'
redshift_port = '5439'

schema_redshift = 'redshift'
schema_athena = 'athena'

table_name_tsv = 'amazon_reviews_tsv'
con_redshift = wr.data_api.redshift.connect(
    cluster_id=redshift_cluster_identifier,
    database=database_name_redshift,
    db_user=master_user_name,
)

In [151]:
statement = """
CREATE EXTERNAL SCHEMA IF NOT EXISTS {} FROM DATA CATALOG 
    DATABASE '{}' 
    IAM_ROLE '{}'
    REGION '{}'
    CREATE EXTERNAL DATABASE IF NOT EXISTS
""".format(schema_athena, database_name_athena, iam_role, region_name)

print(statement)


CREATE EXTERNAL SCHEMA IF NOT EXISTS athena FROM DATA CATALOG 
    DATABASE 'amazon_reviews_capstone' 
    IAM_ROLE 'arn:aws:iam::518606024901:role/Amazon_Reviews_Capstone_Redshift'
    REGION 'us-east-1'
    CREATE EXTERNAL DATABASE IF NOT EXISTS



In [152]:
wr.data_api.redshift.read_sql_query(
    sql=statement,
    con=con_redshift,
)

""


In [153]:
statement = """
SELECT product_category, COUNT(star_rating) AS count_star_rating
    FROM {}.{}
    GROUP BY product_category
    ORDER BY count_star_rating DESC
""".format(schema_athena, table_name_tsv)

print(statement)


SELECT product_category, COUNT(star_rating) AS count_star_rating
    FROM athena.amazon_reviews_tsv
    GROUP BY product_category
    ORDER BY count_star_rating DESC



In [154]:
df = wr.data_api.redshift.read_sql_query(
    sql=statement,
    con=con_redshift,
)

df.head()

,product_category,count_star_rating
0,Books,20726160
1,Digital_Ebook_Purchase,19180765
2,Wireless,9038249
3,Video DVD,7135819
4,PC,7004337


In [156]:
statement = """CREATE SCHEMA IF NOT EXISTS amazon_reviews_capstone""".format(schema_redshift)

wr.data_api.redshift.read_sql_query(
    sql=statement,
    con=con_redshift,
)

""


In [158]:
# Create table function, pass session, table name prefix and start & end year

def create_redshift_table_tsv(wr, con_redshift, table_name_prefix, start_year, end_year):
    for year in range(start_year, end_year + 1, 1):
        current_table_name = table_name_prefix+'_'+str(year)
        statement = """
        CREATE TABLE IF NOT EXISTS redshift.{}( 
             marketplace varchar(2),
             customer_id varchar(8),
             review_id varchar(14),
             product_id varchar(10) DISTKEY,
             product_parent varchar(9),
             product_title varchar(400),
             product_category varchar(24),
             star_rating int,
             helpful_votes int,
             total_votes int,
             vine varchar(1),
             verified_purchase varchar(1),
             review_headline varchar(128),
             review_body varchar(65535),
             review_date varchar(10),
             year int)  SORTKEY (product_category)
        """.format(current_table_name)

        wr.data_api.redshift.read_sql_query(
            sql=statement,
            con=con_redshift,
        )
    print("Done.")

In [159]:
create_redshift_table_tsv(wr, con_redshift, 'amazon_reviews_tsv', 2014, 2015)


Done.


In [160]:
# INSERT INTO function, pass session, table name prefix and start & end year

def insert_into_redshift_table_tsv(wr, con_redshift, table_name_prefix, start_year, end_year):
    for year in range(start_year, end_year + 1, 1):
        print(year)
        current_table_name = table_name_prefix+'_'+str(year)
        statement = """
            INSERT 
            INTO
                redshift.{}
                SELECT
                    marketplace,
                    customer_id,
                    review_id,
                    product_id,
                    product_parent,
                    product_title,
                    product_category,
                    star_rating,
                    helpful_votes,
                    total_votes,
                    vine,
                    verified_purchase,
                    review_headline,
                    review_body,
                    review_date,
                    CAST(DATE_PART_YEAR(TO_DATE(review_date, 'YYYY-MM-DD')) AS INTEGER) AS year
                FROM
                    athena.amazon_reviews_tsv             
                WHERE
                    year = {}
            """.format(current_table_name, year)

        wr.data_api.redshift.read_sql_query(
            sql=statement,
            con=con_redshift,
        )

        df.head()
    print("Done.")

In [ ]:
insert_into_redshift_table_tsv(wr, con_redshift, 'amazon_reviews_tsv', 2014, 2015)

In [162]:
statement = """
SELECT year, product_category, COUNT(star_rating) AS count_star_rating
  FROM redshift.amazon_reviews_tsv_2015
  GROUP BY redshift.amazon_reviews_tsv_2015.product_category, year
UNION ALL
SELECT year, product_category, COUNT(star_rating) AS count_star_rating
  FROM redshift.amazon_reviews_tsv_2014
  GROUP BY redshift.amazon_reviews_tsv_2014.product_category, year
ORDER BY product_category ASC, year DESC
"""

print(statement)


SELECT year, product_category, COUNT(star_rating) AS count_star_rating
  FROM redshift.amazon_reviews_tsv_2015
  GROUP BY redshift.amazon_reviews_tsv_2015.product_category, year
UNION ALL
SELECT year, product_category, COUNT(star_rating) AS count_star_rating
  FROM redshift.amazon_reviews_tsv_2014
  GROUP BY redshift.amazon_reviews_tsv_2014.product_category, year
ORDER BY product_category ASC, year DESC



In [181]:
df = wr.data_api.redshift.read_sql_query(
    sql=statement,
    con=con_redshift,
)

df.head(5)

,year,product_category,count_star_rating
0,2014,Apparel,2122511
1,2014,Automotive,1175637
2,2014,Baby,479630
3,2014,Beauty,1610983
4,2014,Books,3710219


In [182]:
statement = """
SELECT CAST(DATE_PART_YEAR(TO_DATE(review_date, 'YYYY-MM-DD')) AS INTEGER) AS year, product_category, COUNT(star_rating) AS count_star_rating
  FROM athena.amazon_reviews_tsv
  WHERE year = 2014 
  GROUP BY athena.amazon_reviews_tsv.product_category, year
ORDER BY product_category ASC, year DESC
"""

print(statement)


SELECT CAST(DATE_PART_YEAR(TO_DATE(review_date, 'YYYY-MM-DD')) AS INTEGER) AS year, product_category, COUNT(star_rating) AS count_star_rating
  FROM athena.amazon_reviews_tsv
  WHERE year = 2014 
  GROUP BY athena.amazon_reviews_tsv.product_category, year
ORDER BY product_category ASC, year DESC



In [183]:
df = wr.data_api.redshift.read_sql_query(
    sql=statement,
    con=con_redshift,
)

df.head(5)

,year,product_category,count_star_rating
0,2014,Apparel,2122511
1,2014,Automotive,1175637
2,2014,Baby,479630
3,2014,Beauty,1610983
4,2014,Books,3710219


In [184]:
statement = """
SELECT year, product_category, COUNT(star_rating) AS count_star_rating
  FROM redshift.amazon_reviews_tsv_2015
  GROUP BY redshift.amazon_reviews_tsv_2015.product_category, year
UNION ALL
SELECT year, product_category, COUNT(star_rating) AS count_star_rating
  FROM redshift.amazon_reviews_tsv_2014
  GROUP BY redshift.amazon_reviews_tsv_2014.product_category, year
UNION ALL
SELECT CAST(DATE_PART_YEAR(TO_DATE(review_date, 'YYYY-MM-DD')) AS INTEGER) AS year, product_category, COUNT(star_rating) AS count_star_rating
  FROM athena.amazon_reviews_tsv
  WHERE year <= 2013
  GROUP BY athena.amazon_reviews_tsv.product_category, year
ORDER BY product_category ASC, year DESC
"""

print(statement)


SELECT year, product_category, COUNT(star_rating) AS count_star_rating
  FROM redshift.amazon_reviews_tsv_2015
  GROUP BY redshift.amazon_reviews_tsv_2015.product_category, year
UNION ALL
SELECT year, product_category, COUNT(star_rating) AS count_star_rating
  FROM redshift.amazon_reviews_tsv_2014
  GROUP BY redshift.amazon_reviews_tsv_2014.product_category, year
UNION ALL
SELECT CAST(DATE_PART_YEAR(TO_DATE(review_date, 'YYYY-MM-DD')) AS INTEGER) AS year, product_category, COUNT(star_rating) AS count_star_rating
  FROM athena.amazon_reviews_tsv
  WHERE year <= 2013
  GROUP BY athena.amazon_reviews_tsv.product_category, year
ORDER BY product_category ASC, year DESC



In [185]:
df = wr.data_api.redshift.read_sql_query(
    sql=statement,
    con=con_redshift,
)

df.head()

,year,product_category,count_star_rating
0,2014,Apparel,2122511
1,2013,Apparel,854826
2,2012,Apparel,273704
3,2011,Apparel,109328
4,2010,Apparel,57335


In [186]:
statement = """
EXPLAIN 
SELECT year, product_category, COUNT(star_rating) AS count_star_rating
  FROM redshift.amazon_reviews_tsv_2015
  GROUP BY redshift.amazon_reviews_tsv_2015.product_category, year
UNION ALL
SELECT year, product_category, COUNT(star_rating) AS count_star_rating
  FROM redshift.amazon_reviews_tsv_2014
  GROUP BY redshift.amazon_reviews_tsv_2014.product_category, year
UNION ALL
SELECT CAST(DATE_PART_YEAR(TO_DATE(review_date, 'YYYY-MM-DD')) AS INTEGER) AS year, product_category, COUNT(star_rating) AS count_star_rating
  FROM athena.amazon_reviews_tsv
  WHERE year <= 2013
  GROUP BY athena.amazon_reviews_tsv.product_category, year
ORDER BY product_category ASC, year DESC
"""

print(statement)


EXPLAIN 
SELECT year, product_category, COUNT(star_rating) AS count_star_rating
  FROM redshift.amazon_reviews_tsv_2015
  GROUP BY redshift.amazon_reviews_tsv_2015.product_category, year
UNION ALL
SELECT year, product_category, COUNT(star_rating) AS count_star_rating
  FROM redshift.amazon_reviews_tsv_2014
  GROUP BY redshift.amazon_reviews_tsv_2014.product_category, year
UNION ALL
SELECT CAST(DATE_PART_YEAR(TO_DATE(review_date, 'YYYY-MM-DD')) AS INTEGER) AS year, product_category, COUNT(star_rating) AS count_star_rating
  FROM athena.amazon_reviews_tsv
  WHERE year <= 2013
  GROUP BY athena.amazon_reviews_tsv.product_category, year
ORDER BY product_category ASC, year DESC



In [187]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 1024)

df = wr.data_api.redshift.read_sql_query(
    sql=statement,
    con=con_redshift,
)

df.head()

,QUERY PLAN
0,XN Merge (cost=1000176690317.16..1000176690350.73 rows=13426 width=1040)
1,"Merge Key: product_category, ""year"""
2,-> XN Network (cost=1000176690317.16..1000176690350.73 rows=13426 width=1040)
3,Send to leader
4,-> XN Sort (cost=1000176690317.16..1000176690350.73 rows=13426 width=1040)


In [188]:
def unload_redshift_table(wr, con_redshift, table_name_prefix, start_year, end_year, s3_path, iam_role):
    for year in range(start_year, end_year+1, 1):
        current_table_name = table_name_prefix+'_'+str(year)
        statement = """
            UNLOAD ('SELECT marketplace, customer_id, review_id, product_id, product_parent, 
                product_title, product_category, star_rating, helpful_votes, total_votes, 
                vine, verified_purchase, review_headline, review_body, review_date, year 
            FROM redshift.{}')
            TO '{}/{}/'
            IAM_ROLE '{}'
            PARQUET PARALLEL ON 
            PARTITION BY (product_category)
        """.format(current_table_name, s3_path, year, iam_role)

        wr.data_api.redshift.read_sql_query(
            sql=statement,
            con=con_redshift,
        )
    print("Done.")

In [190]:
unload_redshift_table(wr, con_redshift, 'amazon_reviews_tsv', 2014, 2015, "s3://amazon-reviews-capstone/amazon-reviews-pds/parquet-redshift/".format(bucket), iam_role)

RedshiftDataApiTimeoutException: Request ac8e944e-6d49-4a62-ab07-bdd6feb3bf35 timed out after 16 tries and 491.8806266784668s total sleep